<a href="https://colab.research.google.com/github/JordanDCunha/On-Complexity/blob/main/Chapter9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 9.1 Introduction

- Previous cellular automatons (CAs) were **not physical models**; they were not designed to directly represent real-world systems.

- This chapter introduces **cellular automatons used as physical models**, meaning they are intended to describe real physical processes.

- Two main models are explored:
  - **Reaction–diffusion models**  
    - Model chemicals that **diffuse** (spread out) and **react** with each other  
    - Inspired by **Alan Turing’s** idea for explaining how animal patterns (like stripes and spots) form
  - **Percolation models**  
    - Model how a liquid moves through **porous materials**, such as water through coffee grounds

- These models introduce important concepts:
  - **Phase change behavior**
  - **Fractal geometry**

- The code for this chapter is provided in **chap09.ipynb** in the book’s repository.

- Instructions for working with the code are explained in **Section 1.4**.


## 9.2 Diffusion

- Alan Turing (1952) proposed that biological patterns can arise from chemicals that **diffuse** and **react**.
- His original model used **differential equations**, but it can be implemented with a **cellular automaton (CA)**.
- This section begins with a **simpler diffusion model** involving only one chemical.
- A **2-D CA** is used where each cell stores a **continuous value**, usually between **0 and 1**, representing chemical concentration.


In [ ]:
# Kernel for diffusion (difference between a cell and the average of its neighbors)
kernel = np.array([[0, 1, 0],
                   [1, -4, 1],
                   [0, 1, 0]])


## Diffusion Rule

- Each cell is compared to the **average of its neighbors**.
- If the center cell’s concentration is **greater than** the neighborhood average:
  - Chemical flows **from the center to the neighbors**.
- If the center cell’s concentration is **less than** the neighborhood average:
  - Chemical flows **from the neighbors to the center**.


In [ ]:
# Apply the diffusion kernel using 2-D cross-correlation
c = correlate2d(array, kernel, mode='same')


## Updating the CA

- A **diffusion constant r** controls the rate of diffusion.
- The concentration is updated by adding a fraction of the computed differences.


In [ ]:
# Update rule for diffusion
array += r * c


## Model Behavior

- Initial condition:
  - Concentration is 0 everywhere except for a higher-value region in the center.
- Over time:
  - The chemical spreads outward.
  - The system approaches a **uniform concentration**.
- This demonstrates **pure diffusion** without chemical reactions.


## 9.3 Reaction–Diffusion

- This model extends diffusion by adding a **second chemical**.
- Two concentration fields are used:
  - **A**: first chemical
  - **B**: second chemical
- The model combines **diffusion** and **chemical reaction**.
- This type of system was proposed by **Alan Turing** to explain biological pattern formation.


In [ ]:
class ReactionDiffusion(Cell2D):

    def __init__(self, n, m, params, noise=0.1):
        self.params = params
        self.array = np.ones((n, m), dtype=float)
        self.array2 = noise * np.random.random((n, m))
        add_island(self.array2)


## Initialization

- `array` stores the concentration of chemical **A**.
  - Initialized to **1 everywhere**.
- `array2` stores the concentration of chemical **B**.
  - Initialized with small random values.
- An **island** of higher concentration is added to B to break symmetry.


In [ ]:
def add_island(a, height=0.1):
    n, m = a.shape
    radius = min(n, m) // 20
    i = n // 2
    j = m // 2
    a[i-radius:i+radius, j-radius:j+radius] += height


## Step Function

- Each step updates the concentrations of **A** and **B**.
- The update includes:
  - Diffusion of A and B
  - Chemical reaction between A and B
  - Feeding of A
  - Removal (killing) of B


In [ ]:
def step(self):
    A = self.array
    B = self.array2
    ra, rb, f, k = self.params

    cA = correlate2d(A, self.kernel, **self.options)
    cB = correlate2d(B, self.kernel, **self.options)

    reaction = A * B**2
    self.array += ra * cA - reaction + f * (1 - A)
    self.array2 += rb * cB + reaction - (f + k) * B


## Parameters

- **ra**: diffusion rate of A  
- **rb**: diffusion rate of B (usually about half of ra)  
- **f**: feed rate of A  
- **k**: kill rate of B  

- The term `A * B**2` controls the **reaction rate**.
- Reaction:
  - Consumes A
  - Produces B


## Behavior of the Model

- Concentrations of A and B usually remain between **0 and 1**.
- Different values of **f** and **k** produce different patterns:
  - Spots
  - Stripes
  - Coral-like structures
- Many results resemble **animal skin patterns**.


## Observed Patterns

- **f = 0.035, k = 0.057**
  - Light spots of A on dark B background
- **f = 0.055, k = 0.062**
  - Coral-like pattern
- **f = 0.039, k = 0.065**
  - Dividing spots resembling mitosis


## Key Takeaways

- Reaction–diffusion systems combine:
  - Local chemical reactions
  - Spatial diffusion
- Simple rules can generate **complex, natural-looking patterns**.
- Turing’s conjecture is supported by many simulations, though not fully proven.


## 9.4 Percolation — Overview

- Models fluid flow through porous materials (e.g., water in paper, oil in rock)
- Can represent non-physical systems like epidemics and resistor networks
- Implemented here as a 2-D cellular automaton (CA)


In [ ]:
# Percolation CA class definition
class Percolation(Cell2D):

    def __init__(self, n, q):
        self.q = q
        self.array = np.random.choice([1, 0], (n, n), p=[q, 1-q])
        self.array[0] = 5


## Model Rules

- Each cell starts as:
  - **Porous (1)** with probability `q`
  - **Non-porous (0)** with probability `1 - q`
- All cells start **dry** except the top row
- Top row is initialized as **wet (5)**
- Simulation runs until no cells change state
- A *percolating cluster* exists if wet cells connect top to bottom


In [ ]:
# Von Neumann neighborhood kernel (no diagonals)
kernel = np.array([[0, 1, 0],
                   [1, 0, 1],
                   [0, 1, 0]])


## Neighborhood Logic

- Uses a 4-cell **von Neumann neighborhood**
- Kernel sums neighbor states
- If any neighbor is wet (5), the sum ≥ 5
- Porous cells with at least one wet neighbor become wet


In [ ]:
# Step function for percolation update
def step(self):
    a = self.array
    c = correlate2d(a, self.kernel, mode='same')
    self.array[(a == 1) & (c >= 5)] = 5


## Dynamics & Interpretation

- Wet cells spread **only through porous cells**
- Non-porous cells remain dry permanently
- Movement is **up, down, left, right** (no diagonal spread)
- Simulation stops at a fixed point


## Concept Check (Summary)

- `q` = probability a cell is initially porous
- Wet cells:
  - Spread through porous neighbors only
  - Do **not** spread diagonally
- Non-porous cells never become wet


## 9.5 Phase Change — Overview

- Tests whether a random percolation grid contains a percolating cluster
- Identifies a **critical probability** where behavior changes rapidly
- This rapid transition is called a **phase change**


In [ ]:
# Test whether a percolating cluster exists
def test_perc(perc):
    num_wet = perc.num_wet()

    while True:
        perc.step()

        if perc.bottom_row_wet():
            return True

        new_num_wet = perc.num_wet()
        if new_num_wet == num_wet:
            return False

        num_wet = new_num_wet


## Percolation Test Logic

- Advances the CA one step at a time
- Returns **True** if wet cells reach the bottom row
- Stops and returns **False** if a fixed point is reached


In [ ]:
# Estimate probability of percolation
def estimate_prob_percolating(n=100, q=0.5, iters=100):
    t = [test_perc(Percolation(n, q)) for i in range(iters)]
    return np.mean(t)


## Estimating Percolation Probability

- Generates many random percolation grids
- Measures fraction that produce a percolating cluster
- Probability changes rapidly near a critical value of `q`


In [ ]:
# Random walk to estimate critical q
def find_critical(n=100, q=0.6, iters=100):
    qs = [q]
    for i in range(iters):
        perc = Percolation(n, q)
        if test_perc(perc):
            q -= 0.005
        else:
            q += 0.005
        qs.append(q)
    return qs


## Critical Value Estimation

- Adjusts `q` up or down based on percolation outcome
- Produces a sequence of `q` values
- Mean of sequence ≈ **critical probability**
- For n = 100, critical q ≈ **0.59**


## Phase Change Concept

- Near the critical value, system behavior shifts abruptly
- Analogous to physical phase changes (e.g., freezing water)
- Such shared behaviors are called **critical phenomena**


# 9.6 Fractals

## What is Dimension?
- Dimension describes how a quantity **scales with size**
- For simple shapes:
  - Square: area ∝ side² → dimension = 2
  - Cube: volume ∝ side³ → dimension = 3
- The exponent in the scaling law defines the dimension


## Estimating Dimension by Scaling
- Dimension can be estimated by measuring:
  - A notion of **size** (area, volume, number of cells)
  - As a function of a **linear measure** (length, time steps)
- Slope on a **log-log plot** corresponds to dimension


## Example: 1-D Cellular Automata
- Use 1-D CAs to estimate dimension
- Measure:
  - Total number of “on” cells (area)
  - As a function of time steps (rows)
- Compare different CA rules:
  - Rule 20
  - Rule 50
  - Rule 18


In [ ]:
def count_cells(rule, n=500):
    ca = Cell1D(rule, n)
    ca.start_single()

    res = []
    for i in range(1, n):
        cells = np.sum(ca.array)
        res.append((i, i**2, cells))
        ca.step()

    return res


## Interpreting the Log-Log Plots
- Reference lines:
  - Slope = 1 → linear (1-D behavior)
  - Slope = 2 → area-filling (2-D behavior)
- Slopes are estimated from log-log plots of:
  - Number of “on” cells vs time steps


In [ ]:
# Estimated slopes from the log-log plots
rule_20_slope = 1.01
rule_50_slope = 1.97
rule_18_slope = 1.57


## Results by Rule
- **Rule 20**
  - Pattern looks like a line
  - Estimated slope ≈ 1.01
  - Dimension ≈ 1
- **Rule 50**
  - Pattern looks like a filled triangle
  - Estimated slope ≈ 1.97
  - Dimension ≈ 2
- **Rule 18**
  - Pattern is irregular and non-uniform
  - Estimated slope ≈ 1.57
  - Dimension is non-integer


## Fractals and Box-Counting
- Non-integer dimension ⇒ **fractal**
- Rule 18 produces a **fractal pattern**
- This method of estimating dimension is called:
  - **Box-counting**
- Fractals exhibit structure between dimensions, not whole numbers


## Key Takeaways
- Dimension can be estimated from scaling behavior
- Log-log plots reveal dimensionality via slope
- Cellular automata can generate:
  - 1-D structures
  - 2-D structures
  - Fractals with fractional dimension


# 9.7 Fractals and Percolation Models

## Visual Motivation
- Percolation clusters near the critical value visually resemble **fractals**
- As grid size increases (n = 100, 200, 300), clusters show self-similar, irregular structure
- These patterns are neither fully 1-D nor fully 2-D


## Goal: Estimate Fractal Dimension of Percolation Clusters
- Measure how the number of wet cells scales with system size
- Approach:
  - Run percolation simulations at different grid sizes
  - Count wet cells in the percolating cluster
  - Analyze scaling using log-log plots


In [ ]:
res = []
for size in sizes:
    perc = Percolation(size, q)
    if test_perc(perc):
        num_filled = perc.num_wet() - size  # exclude initially wet top row
        res.append((size, size**2, num_filled))


## Data Collected
- Each data point contains:
  - `size`: linear size of the grid
  - `size²`: total number of cells
  - `num_filled`: number of cells in the percolating cluster
- Focus is on how `num_filled` scales with `size`


## Scaling Results
- Plotting number of wet cells vs grid size on a log-log scale:
  - Fitted slope ≈ **1.85** near the critical value of q
- Interpretation:
  - Dimension ≈ 1.85 → **fractal**


## Behavior at Different Values of q
- **q > critical value**
  - Almost all porous cells become wet
  - Wet cells scale as size²
  - Dimension ≈ 2
- **q ≈ critical value**
  - Irregular, branching clusters
  - Wet cells scale with a non-integer power
  - Dimension ≈ 1.85 (fractal)
- **q < critical value**
  - Clusters fail to fill the grid
  - Wet cells scale linearly with size
  - Dimension ≈ 1


## Key Takeaways
- Percolation clusters near the critical point are **fractal**
- Fractal dimension depends on how wet cells scale with system size
- Critical systems often show:
  - Self-similarity
  - Non-integer (fractional) dimensions
- Percolation provides a clear example of fractals emerging from simple rules


# 9.9 Glossary — Key Concepts Summary

## Box-Counting
- Method to estimate **fractal dimension**
- Measures how the number of occupied boxes scales with box size
- Commonly used for irregular or self-similar patterns

## Critical Phenomena
- Shared behaviors observed **near a critical point**
- Includes rapid transitions and scale-invariant structures
- Seen in percolation, phase transitions, and many physical systems

## Diffusion
- Process where substances spread due to particle motion
- Leads to smoothing and equalization of concentrations over time
- Often modeled using cellular automata or differential equations

## Fractal Geometry
- Geometry of **self-similar** patterns
- Parts resemble scaled versions of the whole
- Characterized by **non-integer (fractional) dimensions**
- Common in nature (snowflakes, coastlines, crystals)

## Percolation
- Flow of fluid through a **semi-porous medium**
- Depends on connectivity rather than volume
- Exhibits sharp transitions at a critical probability

## Percolation Model
- Abstract model of percolation processes
- Applied to:
  - Fluid flow
  - Epidemics
  - Electrical networks
- Often studied using cellular automata or random graphs

## Phase Change
- Sudden change in system behavior near a **critical value**
- Example:
  - No percolation → full percolation
- Analogous to physical phase transitions (e.g., freezing)

## Random Walk
- Stochastic process with step-by-step random movement
- Used to estimate critical values and explore state space
- Appears in physics, finance, biology, and algorithms
